In [13]:
import pandas as pd
import json
import simdjson
import os
from nba_api.stats.endpoints import playbyplayv2, commonplayerinfo
from time import sleep

In [14]:
TEAMS_CSV_DIR = "./data/csv/teams"

heights_not_in_api = {
    # Tyler Hansbrough
    201946: "6-9"
}

def get_height_for_player(player_row):
    id = player_row["playerid"]
    # NBA API will block the IP if we request too quickly
    # We need to wait 600ms between requests
    # https://github.com/swar/nba_api/issues/176#issuecomment-771991604
    sleep(0.6)
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=id)
    height_str = player_info.get_normalized_dict()["CommonPlayerInfo"][0]["HEIGHT"]
    if height_str.strip() == "":
        if id in heights_not_in_api:
            height_str = heights_not_in_api[id]
        else:
            raise ValueError(f"No height info for player {id}")
    height_ft, height_in = height_str.split("-")
    height = int(height_ft) * 12 + int(height_in)
    return height

def get_stats_for_team(team_df: pd.DataFrame):
    if "height" not in team_df.columns:
        team_df["height"] = team_df.apply(get_height_for_player, axis="columns")
    return team_df

In [15]:
path = f"{TEAMS_CSV_DIR}/PHI.csv"
print(path)
team_df = pd.read_csv(path)
team_df = get_stats_for_team(team_df)
team_df

./data/csv/teams/PHI.csv
